In [5]:
# Cell 1: Imports and configuration
import pandas as pd
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import Ridge
import warnings
warnings.filterwarnings("ignore")

try:
    from xgboost import XGBRegressor
except Exception:
    XGBRegressor = None

try:
    from lightgbm import LGBMRegressor
except Exception:
    LGBMRegressor = None

@dataclass
class Config:
    random_state: int = 42
    n_splits: int = 5
    growth_min: float = 0.09
    growth_max: float = 0.15
    equal_split_for_multi_tracks: bool = True

CONFIG = Config()
np.random.seed(CONFIG.random_state)
print("Config:", CONFIG)


Config: Config(random_state=42, n_splits=5, growth_min=0.09, growth_max=0.15, equal_split_for_multi_tracks=True)


In [6]:
# Cell 2: Load and combine track CSVs (schema harmonization)
# Expected columns: id, curlingtracks, total, slotDates, createdAt, scannedAt (ignore id/createdAt/scannedAt)

paths = {
    2023: "Final_track_2023_sp.csv",
    2024: "Final_track_2024_fp.csv",
}

frames = []
for yr, p in paths.items():
    df = pd.read_csv(p)
    # Normalize column names just in case
    df.columns = [c.strip() for c in df.columns]
    # Keep only needed columns
    keep_cols = [c for c in ["curlingtracks", "total", "slotDates"] if c in df.columns]
    missing = set(["curlingtracks", "total", "slotDates"]) - set(keep_cols)
    if missing:
        raise ValueError(f"Missing required columns in {p}: {missing}")
    df = df[keep_cols].copy()
    df["year_src"] = yr
    frames.append(df)

raw = pd.concat(frames, ignore_index=True)
print("Loaded rows:", len(raw))
raw.head(3)


Loaded rows: 355


,curlingtracks,total,slotDates,year_src
0,Street Curlingbaan 1,40,2023-11-30 14:00 15:00,2023
1,Street Curlingbaan 1,40,2023-12-07 17:00 18:00,2023
2,Street Curlingbaan 2,40,2023-12-27 17:00 18:00,2023


In [9]:
# Cell 3: Expand comma-separated tracks and distribute totals

def expand_tracks(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # split curlingtracks by comma
    df["curlingtracks"] = df["curlingtracks"].astype(str)
    rows = []
    for _, r in df.iterrows():
        trks = [t.strip() for t in str(r["curlingtracks"]).split(",") if t and t.strip()]
        if len(trks) == 0:
            continue
        if CONFIG.equal_split_for_multi_tracks and len(trks) > 1:
            each_total = r["total"] / len(trks) if pd.notnull(r["total"]) else 0.0
        else:
            each_total = r["total"]
        for t in trks:
            rows.append({
                "curlingtracks": t,
                "total": each_total,
                "slotDates": r["slotDates"],
                "year_src": r.get("year_src", np.nan)
            })
    return pd.DataFrame(rows)

expanded = expand_tracks(raw)
print("Expanded rows:", len(expanded))
expanded.head(5)


Expanded rows: 393


,curlingtracks,total,slotDates,year_src
0,Street Curlingbaan 1,40.0,2023-11-30 14:00 15:00,2023
1,Street Curlingbaan 1,40.0,2023-12-07 17:00 18:00,2023
2,Street Curlingbaan 2,40.0,2023-12-27 17:00 18:00,2023
3,Street Curlingbaan 2,45.0,2023-12-03 15:00 16:00,2023
4,Street Curlingbaan 2,40.0,"2023-12-06 18:00 19:00 , 2023-12-06 18:00 19:00",2023


In [10]:
# Cell 4: Feature engineering and cleaning

def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    fe = df.copy()
    fe["slotDates"] = pd.to_datetime(fe["slotDates"], errors="coerce")
    fe = fe.dropna(subset=["slotDates"]).copy()
    fe["year"] = fe["slotDates"].dt.year
    fe["month"] = fe["slotDates"].dt.month
    fe["weekofyear"] = fe["slotDates"].dt.isocalendar().week.astype(int)
    fe["day"] = fe["slotDates"].dt.day
    fe["dayofweek"] = fe["slotDates"].dt.dayofweek
    fe["is_weekend"] = fe["dayofweek"].isin([5,6]).astype(int)
    fe["is_month_start"] = fe["slotDates"].dt.is_month_start.astype(int)
    fe["is_month_end"] = fe["slotDates"].dt.is_month_end.astype(int)
    return fe

fe = add_time_features(expanded)

# Keep only Nov-Dec for model focus
fe = fe[fe["month"].isin([11, 12])].copy()

# Impute or remove zeros: fill with median by track and weekday
# Compute median per (curlingtracks, dayofweek)
median_map = fe.groupby(["curlingtracks", "dayofweek"])['total'].median().rename('med').reset_index()
fe = fe.merge(median_map, on=["curlingtracks", "dayofweek"], how="left")
# Use mask so we can call fillna on the resulting Series
cond = (fe["total"] <= 0) | (fe["total"].isna())
fe["total"] = fe["total"].mask(cond, fe["med"]).fillna(0)
fe = fe.drop(columns=["med"]) 

# Label encode tracks
le_track = LabelEncoder()
fe["curlingtracks_le"] = le_track.fit_transform(fe["curlingtracks"].astype(str))

# Target transform (log1p)
fe["target_log1p"] = np.log1p(fe["total"].clip(lower=0))

# Feature set
feature_cols = [
    "curlingtracks_le", "year", "month", "weekofyear", "day", "dayofweek",
    "is_weekend", "is_month_start", "is_month_end"
]

X = fe[feature_cols].to_numpy()
y = fe["target_log1p"].to_numpy()
print("Feature shape:", X.shape, "Target shape:", y.shape)
fe.head(3)


Feature shape: (323, 9) Target shape: (323,)


,curlingtracks,total,slotDates,year_src,year,month,weekofyear,day,dayofweek,is_weekend,is_month_start,is_month_end,curlingtracks_le,target_log1p
0,Street Curlingbaan 1,40.0,2023-11-30 15:00:00,2023,2023,11,48,30,3,0,0,1,0,3.713572
1,Street Curlingbaan 1,40.0,2023-12-07 18:00:00,2023,2023,12,49,7,3,0,0,0,0,3.713572
2,Street Curlingbaan 2,40.0,2023-12-27 18:00:00,2023,2023,12,52,27,2,0,0,0,1,3.713572


In [12]:
# Cell 5: Define base models and stacking with OOF meta-features

assert XGBRegressor is not None, "xgboost is required. Please install xgboost."
assert LGBMRegressor is not None, "lightgbm is required. Please install lightgbm."

xgb = XGBRegressor(
    n_estimators=600,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.8,
    random_state=CONFIG.random_state,
    tree_method="hist"
)

lgb = LGBMRegressor(
    n_estimators=1000,
    max_depth=-1,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.8,
    random_state=CONFIG.random_state
)

meta = Ridge(alpha=1.0, random_state=CONFIG.random_state)

ts = TimeSeriesSplit(n_splits=CONFIG.n_splits)

# OOF predictions for meta-learner
oof_xgb = np.zeros(len(fe))
oof_lgb = np.zeros(len(fe))

for fold, (tr_idx, va_idx) in enumerate(ts.split(fe.sort_values("slotDates").index)):
    idx_sorted = fe.sort_values("slotDates").index.to_numpy()
    tr = idx_sorted[tr_idx]
    va = idx_sorted[va_idx]
    X_tr, y_tr = fe.loc[tr, feature_cols].to_numpy(), fe.loc[tr, "target_log1p"].to_numpy()
    X_va, y_va = fe.loc[va, feature_cols].to_numpy(), fe.loc[va, "target_log1p"].to_numpy()

    xgb.fit(X_tr, y_tr)
    lgb.fit(X_tr, y_tr)
    
    oof_xgb[fe.index.get_indexer(va)] = xgb.predict(X_va)
    oof_lgb[fe.index.get_indexer(va)] = lgb.predict(X_va)

meta_X = np.vstack([oof_xgb, oof_lgb]).T
meta_y = y.copy()
meta.fit(meta_X, meta_y)

# Train base models on full data for final inference
xgb.fit(X, y)
lgb.fit(X, y)

# Evaluate stacked model using meta on OOF
pred_oof = meta.predict(meta_X)
# Older sklearn versions don't support squared=False; compute RMSE manually
mse = mean_squared_error(meta_y, pred_oof)
rmse = np.sqrt(mse)
mae = mean_absolute_error(meta_y, pred_oof)
r2 = r2_score(meta_y, pred_oof)
mape = (np.abs(np.expm1(meta_y) - np.expm1(pred_oof)) / np.maximum(1e-9, np.expm1(meta_y))).mean() * 100

metrics = {"rmse": rmse, "mae": mae, "r2": r2, "mape": mape}
metrics


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000019 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18
[LightGBM] [Info] Number of data points in the train set: 58, number of used features: 3
[LightGBM] [Info] Start training from score 3.715556
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

{'rmse': 0.022816316238812466,
 'mae': 0.004138859198111759,
 'r2': 0.004835372054861176,
 'mape': 0.4505408331524424}

In [19]:
# Cell 6: Build forecast calendar (Nov–Dec 2025, Mon–Fri) and predict

# Unique tracks from history
tracks = sorted(fe["curlingtracks"].unique())

# Calendar for weekdays only
cal = pd.date_range(start="2025-11-01", end="2025-12-31", freq="B")  # business days Mon–Fri
cal = cal[(cal.month.isin([11,12]))]

fc_rows = []
for d in cal:
    for t in tracks:
        fc_rows.append({"slotDates": d, "curlingtracks": t})

forecast_df = pd.DataFrame(fc_rows)

# Time features
forecast_df = add_time_features(forecast_df)
# Encode tracks using the same encoder
forecast_df["curlingtracks_le"] = le_track.transform(forecast_df["curlingtracks"].astype(str))

Xf = forecast_df[feature_cols].to_numpy()

# Base model predictions
pred_xgb = xgb.predict(Xf)
pred_lgb = lgb.predict(Xf)
meta_in = np.vstack([pred_xgb, pred_lgb]).T
pred_log = meta.predict(meta_in)

# Baseline total
baseline = np.expm1(pred_log)
forecast_df["baseline"] = baseline

# Apply per-track growth uplift (19%–21.5%)
np.random.seed(CONFIG.random_state)
track_to_growth = {t: np.random.uniform(CONFIG.growth_min, CONFIG.growth_max) for t in tracks}
forecast_df["growth_rate"] = forecast_df["curlingtracks"].map(track_to_growth)
forecast_df["final_forecast"] = forecast_df["baseline"] * (1.0 + forecast_df["growth_rate"]) 

# Final required columns
final_cols = ["curlingtracks", "slotDates", "baseline", "growth_rate", "final_forecast"]
forecast_out = forecast_df[final_cols].copy()

# Constraint: ensure Track 1 > Track 2 by ratio, but cap Track 1 total <= 2300
try:
    track1 = "Street Curlingbaan 1"
    track2 = "Street Curlingbaan 2"
    CAP_TOTAL = 2167.0
    # Historical Nov–Dec 2024 ratio as guidance
    hist_2024 = fe[(fe["year"]==2024) & (fe["month"].isin([11,12])) & (fe["is_weekend"]==0)]
    h1 = hist_2024.loc[hist_2024["curlingtracks"]==track1, "total"].sum()
    h2 = hist_2024.loc[hist_2024["curlingtracks"]==track2, "total"].sum()
    r_hist = (h1 / h2) if (h2 > 0) else 1.2
    r_min = max(1.2, r_hist)
    # Current totals
    f1 = forecast_out.loc[forecast_out["curlingtracks"]==track1, "final_forecast"].sum()
    f2 = forecast_out.loc[forecast_out["curlingtracks"]==track2, "final_forecast"].sum()
    target_min = r_min * max(f2, 1e-9)
    # Determine desired total honoring cap
    if f1 < target_min:
        desired_total = min(target_min, CAP_TOTAL)
    else:
        desired_total = min(f1, CAP_TOTAL)
    if desired_total > 0 and f1 > 0 and not np.isclose(desired_total, f1):
        factor = desired_total / f1
        mask1 = forecast_out["curlingtracks"]==track1
        forecast_out.loc[mask1, "final_forecast"] = forecast_out.loc[mask1, "final_forecast"] * factor
except Exception:
    pass

# Overview
summary = forecast_out.groupby("curlingtracks")[["baseline", "final_forecast"]].sum().reset_index()
summary.head(10)


,curlingtracks,baseline,final_forecast
0,Street Curlingbaan 1,1716.245967,2167.000000
1,Street Curlingbaan 2,1716.271278,1968.636712


In [20]:
# Cell 7: Totals for Nov–Dec 2025 vs 2024 (Mon–Fri)

# 2025 forecast totals (already weekdays only)
forecast_total_2025 = forecast_out["final_forecast"].sum()
forecast_baseline_2025 = forecast_out["baseline"].sum()

# 2024 actual totals (Mon–Fri, Nov–Dec)
fe_2024 = fe[(fe["year"] == 2024) & (fe["month"].isin([11, 12])) & (fe["is_weekend"] == 0)].copy()
actual_total_2024 = fe_2024["total"].sum()

# Comparison summary
comparison = {
    "2024_actual_total": float(actual_total_2024),
    "2025_baseline_total": float(forecast_baseline_2025),
    "2025_forecast_total": float(forecast_total_2025),
    "growth_vs_2024": float((forecast_total_2025 / actual_total_2024 - 1.0) * 100.0) if actual_total_2024 > 0 else None
}
comparison


{'2024_actual_total': 3480.0,
 '2025_baseline_total': 3432.517244261131,
 '2025_forecast_total': 4135.636712003439,
 'growth_vs_2024': 18.840135402397685}

In [21]:
# Cell 8: Save outputs into track/ folder (CSV only, no models)
import os
os.makedirs("track", exist_ok=True)

# Item-level forecast (weekdays Nov-Dec 2025)
items_path = os.path.join("track", "track_forecast_items_nov_dec_2025.csv")
forecast_out.to_csv(items_path, index=False)

# Daily totals per track for 2025
daily_2025 = forecast_out.copy()
daily_2025["date"] = pd.to_datetime(daily_2025["slotDates"]).dt.date
daily_2025 = daily_2025.groupby(["date", "curlingtracks"], as_index=False)["final_forecast"].sum()
daily_path = os.path.join("track", "track_forecast_daily_totals_2025.csv")
daily_2025.to_csv(daily_path, index=False)

# 2024 actual daily (Mon-Fri, Nov-Dec)
fe_2024_daily = fe[(fe["year"] == 2024) & (fe["month"].isin([11, 12])) & (fe["is_weekend"] == 0)].copy()
fe_2024_daily["date"] = fe_2024_daily["slotDates"].dt.date
actual_2024_daily = fe_2024_daily.groupby(["date", "curlingtracks"], as_index=False)["total"].sum()
actual_2024_daily_path = os.path.join("track", "track_actual_daily_totals_2024.csv")
actual_2024_daily.to_csv(actual_2024_daily_path, index=False)

# Summary totals
summary_totals = pd.DataFrame({
    "metric": ["2024_actual_total", "2025_baseline_total", "2025_forecast_total", "growth_vs_2024"],
    "value": [comparison["2024_actual_total"], comparison["2025_baseline_total"], comparison["2025_forecast_total"], comparison["growth_vs_2024"]]
})
summary_path = os.path.join("track", "track_summary_totals.csv")
summary_totals.to_csv(summary_path, index=False)

print("Saved:")
print("-", items_path)
print("-", daily_path)
print("-", actual_2024_daily_path)
print("-", summary_path)


Saved:
- track\track_forecast_items_nov_dec_2025.csv
- track\track_forecast_daily_totals_2025.csv
- track\track_actual_daily_totals_2024.csv
- track\track_summary_totals.csv
